In [1]:
!gdown 1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
!gdown 1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1

Downloading...
From: https://drive.google.com/uc?id=1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
To: /content/test_data.csv
100% 565k/565k [00:00<00:00, 8.21MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1
To: /content/train_data.csv
100% 2.24M/2.24M [00:00<00:00, 18.8MB/s]


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import MinMaxScaler

In [3]:
train_df = pd.read_csv("train_data.csv")
train_df = train_df[train_df['confidence'] >= 0.8].reset_index(drop=True)

In [4]:
test_df = pd.read_csv("test_data.csv")
test_df = test_df[test_df['confidence'] >= 0.8].reset_index(drop=True)

In [5]:
print(train_df.shape, test_df.shape)

(1832, 111) (465, 111)


In [6]:
X_train = train_df.drop(['target'], axis=1)
y_train = train_df['target'].to_numpy()
X_test = test_df.drop(['target'], axis=1)
y_test = test_df['target'].to_numpy()

# Bag Of Words

In [7]:
X_train_bow = X_train.drop(['clean_text'], axis=1).to_numpy()
X_test_bow = X_test.drop(['clean_text'], axis=1).to_numpy()

In [8]:
cv = CountVectorizer(max_features = 5000)

In [9]:
cv.fit(pd.concat([X_train['clean_text'], X_test['clean_text']],ignore_index=True))

CountVectorizer(max_features=5000)

In [10]:
train_bow = cv.transform(X_train['clean_text']).toarray()

In [11]:
X_train_bow = np.concatenate((X_train_bow, train_bow), axis=1)

In [12]:
test_bow = cv.transform(X_test['clean_text']).toarray()

In [13]:
X_test_bow = np.concatenate((X_test_bow, test_bow), axis=1)

In [14]:
print(X_train_bow.shape)
print(X_test_bow.shape)

(1832, 5109)
(465, 5109)


# Training-Testing Phase

In [15]:
def training_and_prediction(training_model, X_train, X_test, y_train, y_test):
  model = training_model
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  # print("Accuracy =", accuracy)
  # print("Confusion Matrix =\n", confusion_matrix(y_test, y_pred))
  # print(classification_report(y_test, y_pred))
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

In [16]:
def get_report(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

## Naive Bayes

In [ ]:
scaler = MinMaxScaler()
training_and_prediction(MultinomialNB(alpha=0.7), scaler.fit_transform(X_train_bow), scaler.fit_transform(X_test_bow), y_train, y_test)

{'Accuracy': 0.7913978494623656,
 'Precision': 0.7638376383763837,
 'Recall': 0.8625,
 'F1-Score': 0.8101761252446185}

In [ ]:
training_and_prediction(GaussianNB(), X_train_bow, X_test_bow, y_train, y_test)

{'Accuracy': 0.6924731182795699,
 'Precision': 0.6802973977695167,
 'Recall': 0.7625,
 'F1-Score': 0.719056974459725}

In [ ]:
training_and_prediction(BernoulliNB(), X_train_bow, X_test_bow, y_train, y_test)

{'Accuracy': 0.789247311827957,
 'Precision': 0.7689393939393939,
 'Recall': 0.8458333333333333,
 'F1-Score': 0.8055555555555556}

## SVM

In [ ]:
training_and_prediction(SVC(kernel='poly'), X_train_bow, X_test_bow, y_train, y_test)

{'Accuracy': 0.7935483870967742,
 'Precision': 0.7748091603053435,
 'Recall': 0.8458333333333333,
 'F1-Score': 0.8087649402390439}

## Random Forest

In [ ]:
training_and_prediction(RandomForestClassifier(), X_train_bow, X_test_bow, y_train, y_test)

{'Accuracy': 0.8086021505376344,
 'Precision': 0.805668016194332,
 'Recall': 0.8291666666666667,
 'F1-Score': 0.8172484599589322}

## Logistic Regression

In [ ]:
training_and_prediction(LogisticRegression(max_iter=5000, solver='saga'), X_train_bow, X_test_bow, y_train, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


{'Accuracy': 0.8064516129032258,
 'Precision': 0.8073770491803278,
 'Recall': 0.8208333333333333,
 'F1-Score': 0.8140495867768595}

## Gredient Boosting

In [ ]:
training_and_prediction(XGBClassifier(), X_train_bow, X_test_bow, y_train, y_test)

{'Accuracy': 0.8172043010752689,
 'Precision': 0.8242677824267782,
 'Recall': 0.8208333333333333,
 'F1-Score': 0.8225469728601252}

## Deep Learning Techniques with Word2vec

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Bidirectional, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

### ANN

In [18]:
def ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate=0):
    model_dense = Sequential()
    model_dense.add(Dense(num_neurons_layer1, activation='relu', input_shape=(X_train_bow.shape[1],)))
    model_dense.add(Dropout(dropout_rate))
    model_dense.add(Dense(num_neurons_layer2, activation='relu'))
    model_dense.add(Dense(1, activation='sigmoid'))
    model_dense.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_dense.fit(X_train_bow, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_bow, y_test))
    y_pred_dense = model_dense.predict(X_test_bow)
    y_pred_dense = (y_pred_dense > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'num_neurons_layer2': num_neurons_layer2, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_dense)]

In [19]:
for num_neurons_layer1 in [128, 64]:
  for num_neurons_layer2 in [64, 32]:
    for dropout_rate in [0.0, 0.1, 0.2, 0.3, 0.4]:
      print(ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate))

15/15 [==============================] - 0s 7ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.0}, {'Accuracy': 0.8064516129032258, 'Precision': 0.7798507462686567, 'Recall': 0.8708333333333333, 'F1-Score': 0.8228346456692913}]
15/15 [==============================] - 0s 3ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.1}, {'Accuracy': 0.8064516129032258, 'Precision': 0.8177966101694916, 'Recall': 0.8041666666666667, 'F1-Score': 0.8109243697478992}]
15/15 [==============================] - 0s 2ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.2}, {'Accuracy': 0.8, 'Precision': 0.8237885462555066, 'Recall': 0.7791666666666667, 'F1-Score': 0.8008565310492504}]
15/15 [==============================] - 0s 2ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.3}, {'Accuracy': 0.8150537634408602, 'Precision': 0.7961538461538461, 'Recall': 0.8625, 'F1-Score': 0.828}]
15/1